In [17]:
# Install DuckDB
!pip install -q duckdb


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [18]:
import duckdb
duckdb.__version__

'1.4.3'

In [19]:
con = duckdb.connect("demo.duckdb")

In [20]:
con.execute("""
INSTALL httpfs;
LOAD httpfs;
""")

In [21]:
con.execute("""

CREATE OR REPLACE TABLE eligibility AS
SELECT *
FROM read_csv_auto(
  'https://tuva-public-resources.s3.us-east-1.amazonaws.com/versioned_tuva_synthetic_data/0.15.0/eligibility.csv'
);

CREATE OR REPLACE TABLE medical_claim_src AS
SELECT *
FROM read_csv_auto(
  'https://tuva-public-resources.s3.us-east-1.amazonaws.com/versioned_tuva_synthetic_data/0.15.0/medical_claim.csv'
);

CREATE OR REPLACE TABLE pharmacy_claim AS
SELECT *
FROM read_csv_auto(
  'https://tuva-public-resources.s3.us-east-1.amazonaws.com/versioned_tuva_synthetic_data/0.15.0/pharmacy_claim.csv'
);

""")

In [22]:
con.execute("""
select count(1)
from pharmacy_claim

""").fetchall()

[(12019,)]

In [23]:
con.execute("""

-- Select your medical_claim table here:
with medical_claim as (
select *
from medical_claim_src
),

-- Here we filter to all ED claims:
-- (Definition of an ED claim)
all_ed_inst_claims as (
select distinct claim_id
from medical_claim
where (substring(hcpcs_code, 1, 4) = '9928') or
(substring(hcpcs_code, 1, 4) = 'G038') or
revenue_center_code in ('0450','0451','0452','0456','0459','0981')
),

-- We define dates used for merging ED claims into encounters:
ed_claims_with_merge_dates as (
select 
  aa.claim_id as claim_id,
  max(aa.person_id) as person_id,
  max(aa.claim_type) as claim_type,
  coalesce(  min(aa.admission_date),
             min(aa.claim_start_date),
	     min(aa.claim_line_start_date)  ) as merge_start_date,
  coalesce(  max(aa.discharge_date),
             max(aa.claim_end_date),
	     max(aa.claim_line_end_date)  ) as merge_end_date
from medical_claim aa
inner join all_ed_inst_claims bb
on aa.claim_id = bb.claim_id
group by aa.claim_id
),

-- ******************************************************************
-- The following CTEs provide the logic for merging individual claims
-- into encounters:
-- ******************************************************************
add_row_num as (
select
  person_id,
  claim_id,
  claim_type,
  merge_start_date,
  merge_end_date,
  row_number() over (partition by person_id
                     order by merge_end_date, merge_start_date, claim_id) as row_num
from ed_claims_with_merge_dates
),

check_for_merges_with_larger_row_num as (
select
  aa.person_id,
  aa.claim_id as claim_id_a,
  bb.claim_id as claim_id_b,
  aa.row_num as row_num_a,
  bb.row_num as row_num_b,

  case
    -- Claims that overlap are merged
    -- into the same encounter:
    when
    (
      (aa.merge_start_date between bb.merge_start_date and bb.merge_end_date)
      or
      (aa.merge_end_date between bb.merge_start_date and bb.merge_end_date)
      or
      (bb.merge_start_date between aa.merge_start_date and aa.merge_end_date)
      or
      (bb.merge_end_date between aa.merge_start_date and aa.merge_end_date)
    ) then 1
    else 0
  end as merge_flag

from add_row_num aa
     inner join add_row_num bb
     on aa.person_id = bb.person_id
     and aa.row_num < bb.row_num
),

merges_with_larger_row_num as (
select
  person_id,
  claim_id_a,
  claim_id_b,
  row_num_a,
  row_num_b,
  merge_flag
from check_for_merges_with_larger_row_num
where merge_flag = 1
),

claim_ids_that_merge_with_larger_row_num as (
select distinct claim_id_a as claim_id
from merges_with_larger_row_num
),

claim_ids_having_a_smaller_row_num_merging_with_a_larger_row_num as (
select distinct aa.claim_id as claim_id
from add_row_num aa
     inner join
     merges_with_larger_row_num bb
     on aa.person_id = bb.person_id
     and bb.row_num_a < aa.row_num
     and bb.row_num_b > aa.row_num
),

close_flags as (
select
  aa.person_id,
  aa.claim_id,
  aa.merge_start_date,
  aa.merge_end_date,
  aa.row_num,
  case when (bb.claim_id is null and cc.claim_id is null) then 1
       else 0
  end as close_flag

from add_row_num aa

left join claim_ids_that_merge_with_larger_row_num bb
on aa.claim_id = bb.claim_id

left join claim_ids_having_a_smaller_row_num_merging_with_a_larger_row_num cc
on aa.claim_id = cc.claim_id
),

join_every_row_to_later_closes as (
select
  aa.person_id as person_id,
  aa.claim_id as claim_id,
  aa.row_num as row_num,
  bb.row_num as row_num_b
from close_flags aa inner join close_flags bb
     on aa.person_id = bb.person_id
     and aa.row_num <= bb.row_num
where bb.close_flag = 1
),

find_min_closing_row_num_for_every_claim as (
select
  person_id,
  claim_id,
  min(row_num_b) as min_closing_row
from join_every_row_to_later_closes
group by person_id, claim_id
),

add_min_closing_row_to_every_claim as (
select
  aa.person_id as person_id,
  aa.claim_id as claim_id,
  aa.merge_start_date as merge_start_date,
  aa.merge_end_date as merge_end_date,
  aa.row_num as row_num,
  aa.close_flag as close_flag,
  bb.min_closing_row as min_closing_row
from close_flags aa
     left join find_min_closing_row_num_for_every_claim bb
     on aa.person_id = bb.person_id
     and aa.claim_id = bb.claim_id
),

add_encounter_id as (
select
  aa.person_id as person_id,
  aa.claim_id as claim_id,
  aa.merge_start_date as merge_start_date,
  aa.merge_end_date as merge_end_date,
  aa.row_num as row_num,
  aa.close_flag as close_flag,
  aa.min_closing_row as min_closing_row,
  bb.claim_id as encounter_id
from add_min_closing_row_to_every_claim aa
     left join add_min_closing_row_to_every_claim bb
     on aa.person_id = bb.person_id
     and aa.min_closing_row = bb.row_num
),

ed_encounters as (
select
  person_id,
  claim_id,
  encounter_id
from add_encounter_id
)

select *
from add_encounter_id

""").df()

,person_id,claim_id,merge_start_date,merge_end_date,row_num,close_flag,min_closing_row,encounter_id
0,10718,163248,2018-03-20,2018-03-20,3,0,4,1201516
1,12154,1251199,2018-08-05,2018-08-05,3,1,3,1251199
2,12500,951237,2017-09-04,2017-09-04,2,1,2,951237
3,11862,555771,2017-10-18,2017-10-18,4,1,4,555771
4,11594,998891,2018-07-25,2018-07-25,1,1,1,998891
...,...,...,...,...,...,...,...,...
1621,12621,541786,2017-12-26,2017-12-26,5,1,5,541786
1622,1202,716867,2018-02-08,2018-02-08,6,1,6,716867
1623,1202,721119,2018-07-25,2018-07-25,7,1,7,721119
1624,133,311182,2017-09-28,2017-09-28,15,1,15,311182
